In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.io.fits as fits
import ipywidgets
import yaml

import batoid
import batoid_rubin

%matplotlib widget

In [ ]:
def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

In [ ]:
# Read field XY
with open("fieldXY.yaml") as f:
    data = yaml.safe_load(f)
field_x = np.array(data['x'])
field_y = np.array(data['y'])

In [ ]:
telescope = batoid.Optic.fromYaml("LSST_g_500.yaml")
wavelength = 500e-9

In [ ]:
@ipywidgets.interact(
     i=ipywidgets.BoundedIntText(value=0, min=0, max=34)
)
def f(i):
    ts_phosim_opd = fits.getdata(f"phosim/opd_nominal_field_{i}.fits.gz")
    
    # Convert from batoid -> phosim.
    # Implies flipping input theta_x and fliplr the output image
    batoid_opd = batoid.wavefront(
        telescope,
        -np.deg2rad(field_x[i]),
        np.deg2rad(field_y[i]),
        wavelength, nx=255, 
    )
    batoid_opd.array = np.fliplr(batoid_opd.array)
    # batoid in waves => microns
    batoid_opd.array *= wavelength*1e6
    
    vmax = np.quantile(np.abs(batoid_opd.array), 0.9)
    
    fig, axes = plt.subplots(ncols=3, figsize=(8, 3))
    colorbar(axes[0].imshow(ts_phosim_opd, vmin=-vmax, vmax=vmax, cmap='seismic'))
    axes[0].set_title("ts_phosim")

    colorbar(axes[1].imshow(batoid_opd.array, vmin=-vmax, vmax=vmax, cmap='seismic'))
    axes[1].set_title("batoid")
    
    colorbar(axes[2].imshow(batoid_opd.array - ts_phosim_opd, vmin=-0.01*vmax, vmax=0.01*vmax, cmap='seismic'))
    axes[2].set_title("b - ph")

    for ax in axes:
        ax.set_aspect('equal')
    fig.tight_layout()
    plt.show()